### 2020-07-27T02:12:40Z という形式で出力する方法
### pandasの時系列カラムの時刻を特定の書式文字列に変換する方法

https://ufirst.jp/memo/2020/07/27/post-2638/  
2020-07-27T02:12:40Z  
という時刻形式がある。

ISO規格だと ISO 8601  
RFC規格だとRFC 3339 （ほぼ同じと見ていいらしい。Wikipediaの情報だけど）

pandasの時刻データをこの形で作る方法。  
何でか? サンプルデータを作りたくて、その時刻形式がこれだったからです。

後で読む
https://teratail.com/questions/249408

最近発生した状況。

* データ分析用にダミーの簡単なデータを作る必要がある
* そのデータは時刻カラムを含む
* 時刻カラムは、タイムゾーンが設定されていて、UTCである
* 実際のデータは表示書式はYYYY-MM-DDThh:mm:ssZ の形式 （例 2020-07-27T02:12:40Z）であるため、ダミーデータについても同じ書式で作成したい
* どうすれば実現できるか?

注意：以下の説明で、「time zone naive = タイムゾーンが設定されていない」「time zone aware = タイムゾーンが設定されている」という意味である。

In [1]:
import pandas as pd
import datetime
# pd.options.display.notebook_repr_html = False  # jupyter notebook上での出力形式を制御するために書いています。無くても動きます。

In [2]:
# 動作環境の確認
print(pd.__version__)

1.1.2


時刻の列として、適当な時刻を3つほど作成する。  datetime オブエジェクトを作り、日付と時までを適当に埋めよう。

In [3]:
datetime_list = [
    datetime.datetime(2022, 1, 2, 3, 0),
    datetime.datetime(2022, 2, 3, 4, 0),
    datetime.datetime(2022, 3, 4, 5, 0),
]

In [4]:
val_list = [10, 30, 20,]

In [5]:
df_datetime = pd.DataFrame({
    'datetime'    : datetime_list,
    'val' : val_list
})

In [6]:
df_datetime

,datetime,val
0,2022-01-02 03:00:00,10
1,2022-02-03 04:00:00,30
2,2022-03-04 05:00:00,20


自分の理解の整理のために、Q&Aの形式で書いていく。

Q1. このDataFrameのdatetimeカラムは、タイムゾーンがある時刻か、ない時刻か?  
A1. タイムゾーンがない時刻である。

Q2. タイムゾーンがないということはどうして分かるのか?  
A2. 以下2つの方法がある。

1つ目の方法は、カラムを調べることである。Series（カラム）にタイムゾーンがないことは、Seriesのdtypeを見れば分かる。

> https://pandas.pydata.org/docs/user_guide/timeseries.html#time-zone-series-operations
> A Series with time zone naive values is represented with a dtype of datetime64[ns].
> A Series with a time zone aware values is represented with a dtype of datetime64[ns, tz] where tz is the time zone.

拙訳：

In [7]:
df_datetime.dtypes

datetime    datetime64[ns]
val                  int64
dtype: object

2つ目の方法は、入っている時刻データを調べることである。

https://pandas.pydata.org/docs/user_guide/timeseries.html#time-zone-handling

> ほげ
> ほげ

拙訳：

In [8]:
datetime1 = df_datetime.loc[0, 'datetime']
datetime1

Timestamp('2022-01-02 03:00:00')

In [9]:
# 注：datetime1.tzと単に書くと、jupyter notebook上で結果のNoneが表示されないので、明示的にprintをつけてNoneを表示させている。
print(datetime1.tz)

None


どちらにせよ、タイムゾーンが設定されていないことが分かった。

さて、欲しいデータはUTCなので、タイムゾーンを設定しよう。

In [10]:
# dateカラムはタイムゾーンがあるかない→ない（なぜならXXX）
# タイムゾーンを設定します

In [11]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.tz_localize.html

In [12]:

df_datetime['datetime_utc'] = df_datetime['datetime'].tz_localize(tz='UTC')

# これはダメ。インデックスを変換しようとするから。

# Q：何でエラーなの


TypeError: index is not a valid DatetimeIndex or PeriodIndex

In [13]:
# Q ただのtz_localizeと dt.tz_localize って何が違うの
# Q dtって何

# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.tz_localize.html#pandas.Series.dt.tz_localize
df_datetime['datetime_utc'] = df_datetime['datetime'].dt.tz_localize(tz='UTC')


In [14]:
df_datetime

,datetime,val,datetime_utc
0,2022-01-02 03:00:00,10,2022-01-02 03:00:00+00:00
1,2022-02-03 04:00:00,30,2022-02-03 04:00:00+00:00
2,2022-03-04 05:00:00,20,2022-03-04 05:00:00+00:00


In [15]:
# 2021-01-11 00:00:00+00:00 となってしまった。
# 2021-01-11T00:00:00Z が良いんだけど。

In [16]:
df_datetime.dtypes

datetime             datetime64[ns]
val                           int64
datetime_utc    datetime64[ns, UTC]
dtype: object

2つの方法が考えられる。

* 時刻のデータ（dtype　`datetime64[ns, UTC]`）のまま、表示方法を変更する。
* 時刻から希望する形式の文字列に変換する。


In [17]:
# Q 時刻を表示する形式を変更する方法はあるのか?
# A 多分ない

# Q ではどうすれば所望の文字列を得られるのか?
# Q typeはどうなってるのか?